<h1><center>IBM Data Science Capstone Project Final Submission</center></h1>

<h4> <center>This is the final submission in the IBM Data Science Capstone Project</center> </h4>

<h2> Week 4 Submission <h2> 

<h3>Business Understanding</h3>
<h5> For this project we are doing a case study on an Italian immigrant moving to Canada. He is a entrepreneur who want to start an Italian restaurant in Canada that serves authentic home cooked style meal rather than commercial chain.
    <p>The requirements that were specified by the client are as follows, </p>
    <ol>
        <li>Client wants to open his restaurant in one of the five largest city by Population. </li>
        <li>Client wants to know the open his restaurant in or near downtown area.</li>
        <li>Client wants to know the Italian restaurants nearby to gage competition.</li>
        <li>Client wants to know how the existing Italian restaurant are performing. </li>
        <li>Client wants to know the Italian (country of origin) population as he will be serving Italian home cooked meal.</li>        
        <li>Client wants to know the estimated rental price of the restaurant place (area is approximately 1000 -1500 sq.ft).</li>
        <li>Client wants to know the crime rate in the downtown areas to know if its safe.</li>
    </ol>

<h3>Analytic Approach</h3>
<h5>From the requirements given by the client, our job is to find a city and place where our client can successfully open and run his Italian restaurant. For this we need to collect data from various sources on the population data,and people of Italian origin , what areas constitute as downtown, crime data in these area, find the trends and like for restaurants in the downtown are and lastly estimate the rental price for a space 1000 sq.ft to 1500 sq.ft
The approach chosen is as follows:
    <ol>
        <li>Collect population size of 5 largest cities in Canada and population demographics for Italian origin people. This will give us insight into possible customer flow for our client.</li>
        <li>List of neighborhood in these cities in the downtown.Use clustering method.</li>
        <li>Get the crime rate data for these areas. Use clustering method to group crimes.</li>
        <li>Existing Italian restaurant in these neighborhood and like they received. Use statistical method.</li>
        <li>Estimate rental prices based on square foot area and estimate the price for restaurant.Use statistical method use linear regression</li>        
    </ol>
</h5>

<h3>Data Requirement</h3>
<h5>The required data for this analysis 
    <ol>
        <li>Population and demographic data for various cities in Canada</li>
        <li>List of neighborhoods in cities in Canada</li>
        <li>Data on crime rates from the given cities</li>
        <li>Data on Italian restaurant and likes they received </li>
        <li>Data on rental price of commercial location in downtown of these cities </li>
    </ol></h5>

<body>
    <h3> Data Collection </h3>
    <h5><p>Since the client asked variety of data and there is no single source for collecting such data. Various sources are available to get data. Here is are some of the sources of data</p>
    </h5>
       <table>
            <tr><th>Data Required</th>
                <th colspan = "1">Data Source</th>
                <th colspan = "1">Link </th>
            </tr>
            <tr>            
                <td>Population and Demographics</td>
                <td> Wikipedia Page </td>
                <td> <a href = "https://en.wikipedia.org/wiki/Demographics_of_Canada">Wikipedia page</a></td>
            </tr>   
            <tr>            
                <td>Downtown Neighborhoods</td>
                <td> Wikipedia Page </td>
                <td> <a href = "https://en.wikipedia.org/wiki/Downtown_Toronto">Wikipedia page</a></td>
            </tr> 
            <tr>            
                <td>Crime Rate</td>
                <td> Open Data from Various Cities </td>
                <td> <a href = "https://data.calgary.ca/">Calgary open data</a></td>
            </tr>        
            <tr>            
                <td>Venues and likes in downtown of cities</td>
                <td> Foursquare API </td>
                <td> <a href = "https://developer.foursquare.com/docs/api/endpoints">Foursquare page</a></td>
            </tr> 
            <tr>            
                <td>Rental Prices from downtown to various cities</td>
                <td> Rental Websites like Space lease, kijiji </td>
                <td> <a href = "https://www.spacelist.ca/listings/ab/calgary">Spacelist page</a></td>
            </tr>         
         </table>
<body>
        

In [1]:
#import various libraries
import pandas as pd # this is important in any analysis
import requests as rq # this is imported to request the website data
import numpy as np #for any data science analysis


!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forge
    lib

In [2]:
!pip install beautifulsoup4
!pip install lxml

     |████████████████████████████████| 112kB 8.5MB/s eta 0:00:01
     |████████████████████████████████| 5.8MB 31.3MB/s eta 0:00:01


In [3]:
# get population on various cities in canada
population_page = rq.get("https://www.worldatlas.com/articles/biggest-cities-in-canada.html")
print("status of the population page is:" ,population_page.status_code)

#get table information from this page
from bs4 import BeautifulSoup
pop_soup = BeautifulSoup(population_page.text, 'html.parser')

status of the population page is: 200


In [4]:
header = pop_soup.find('tr')
colname = [col.get_text() for col in header.find_all('th')]
colname = ['Rank', 'City', 'Population']
df_pop = pd.DataFrame(columns=colname)
df_pop

,Rank,City,Population


In [5]:
n_rows = []
Areas  = pop_soup.find_all('tr')
for Area in Areas:
    rows = [row.get_text() for row in Area.find_all('td')]
    n_rows.append(rows)
    df_pop = pd.DataFrame(n_rows)

df_pop.columns = colname
df_pop.dropna(axis=0,inplace=True)
print(df_pop.columns)

Index(['Rank', 'City', 'Population'], dtype='object')


In [6]:
#df_pop['City'] = df_pop['City'].replace(","," ",inplace = True)
df_pop['Name'] = df_pop.City.str.split(",")
df_pop['City'] = df_pop.City.str.split(",", expand=True)
df_pop = df_pop.drop(['Name'], axis=1)
df_pop = df_pop.replace(' ','',regex=True)

<h5>Get the people from Italian country of origin</h5>

In [7]:
#demographics page
demo_page = rq.get("https://en.wikipedia.org/wiki/Italian_Canadians")
print("page status for Italian Canadians:", demo_page.status_code)

demo_soup = BeautifulSoup(demo_page.text,'html.parser')

page status for Italian Canadians: 200


In [8]:
demo_table = demo_soup.findAll('table')[3]

In [9]:
demo_header =  demo_table.find('tr')
colname1 = [col.get_text() for col in demo_header.find_all('th')]
for col in colname1:
    coln = col.strip("\n")

df_demo = pd.DataFrame(columns=colname1)
colname1[7] = 'Italian Population (2016)'
colname1[8] = '% of ethnic population(2016)'
colname1[0] = 'City'
df_demo

,Metropolitan area\n,Population (2001)[23]\n,% of ethnic population (2001)\n,Population (2006)[24]\n,% of ethnic population (2006)\n,Population (2011)[25]\n,% of ethnic population (2011)\n,Population (2016)[26]\n,% of ethnic population (2016)\n


In [10]:
demo_rows = []
dems  = demo_table.find_all('tr')
for dem in dems:
    rows = [row.get_text() for row in dem.find_all('td')]
    demo_rows.append(rows)
    df_demo = pd.DataFrame(demo_rows)

#df_pop.columns = colname
df_demo.dropna(axis=0,inplace=True)
df_demo = df_demo.replace('\n','',regex=True)
df_demo = df_demo.replace('CMA','',regex=True)
df_demo.columns = colname1
df_demo

,City,Population (2001)[23]\n,% of ethnic population (2001)\n,Population (2006)[24]\n,% of ethnic population (2006)\n,Population (2011)[25]\n,% of ethnic population (2011)\n,Italian Population (2016),% of ethnic population(2016)
1,Toronto [note 2],"429,380[note 3]",9.2%,"466,155[note 4]",9.2%,"475,090[note 5]",8.6%,"484,360[note 6]",8.3%
2,Montreal,"224,460",6.6%,"260,345",7.3%,"263,565",7.0%,"279,795",7.0%
3,Greater Vancouver,"69,000",3.5%,"76,345",3.6%,"82,435",3.6%,"87,875",3.6%
4,Hamilton,"67,685[note 7]",10.3%,"72,440[note 8]",10.6%,"75,900[note 9]",10.7%,"79,725[note 10]",10.8%
5,National Capital Region,"37,435",3.6%,"45,005",4.0%,"47,975",4.0%,"53,825",4.1%
6,Niagara Region,"44,645",12.0%,"48,850",12.7%,"48,530",12.6%,"49,345",12.4%
7,Greater Calgary,"29,120",3.1%,"33,645",3.1%,"36,875",3.1%,"42,940",3.1%
8,Greater Edmonton,"22,385",2.4%,"28,805",2.8%,"29,580",2.6%,"33,800",2.6%
9,Windsor,"30,680",10.1%,"33,725",10.5%,"30,880",9.8%,"33,175",10.2%
10,Oshawa,"13,990[note 11]",4.8%,"18,225[note 12]",5.6%,"20,265[note 13]",5.8%,"22,870[note 14]",6.1%


<h5>We need to select data from this above table to create 2016 population data </h5>

In [11]:
df_demo = df_demo.drop(df_demo.columns[1:7],axis=1)
df_demo

,City,Italian Population (2016),% of ethnic population(2016)
1,Toronto [note 2],"484,360[note 6]",8.3%
2,Montreal,"279,795",7.0%
3,Greater Vancouver,"87,875",3.6%
4,Hamilton,"79,725[note 10]",10.8%
5,National Capital Region,"53,825",4.1%
6,Niagara Region,"49,345",12.4%
7,Greater Calgary,"42,940",3.1%
8,Greater Edmonton,"33,800",2.6%
9,Windsor,"33,175",10.2%
10,Oshawa,"22,870[note 14]",6.1%


<h5> We need to make some changes in the dataframe, remove \n from columns, remove the 'note' from dataframe.</h5>
<h5> Also the national capital region is actually Ottowa</h5>

In [12]:
df_demo = df_demo.replace('National Capital Region','Ottowa', regex=True)
df_demo = df_demo.replace('Greater','', regex=True)

In [13]:
df_demo = df_demo.replace('note 2','', regex=True)
df_demo = df_demo.replace('note 6','', regex=True)
df_demo = df_demo.replace('note 10','', regex=True)
df_demo = df_demo.replace('note 14','', regex=True)
df_demo = df_demo.replace('\\[','', regex=True)
df_demo = df_demo.replace('\\]','', regex=True)
df_demo = df_demo.replace(' ','', regex=True)

In [14]:
df_demo

,City,Italian Population (2016),% of ethnic population(2016)
1,Toronto,"484,360",8.3%
2,Montreal,"279,795",7.0%
3,Vancouver,"87,875",3.6%
4,Hamilton,"79,725",10.8%
5,Ottowa,"53,825",4.1%
6,NiagaraRegion,"49,345",12.4%
7,Calgary,"42,940",3.1%
8,Edmonton,"33,800",2.6%
9,Windsor,"33,175",10.2%
10,Oshawa,"22,870",6.1%


<h4>Now lets match the Italian Origin Population with the total population of the cities</h4>

In [15]:
df_compare = pd.merge(df_pop,df_demo, how = 'inner', left_on=['City'], right_on=['City'])
df_compare.head(5)

,Rank,City,Population,Italian Population (2016),% of ethnic population(2016)
0,1,Toronto,"5,429,524","484,360",8.3%
1,2,Montreal,"3,519,595","279,795",7.0%
2,3,Vancouver,"2,264,823","87,875",3.6%
3,4,Calgary,"1,237,656","42,940",3.1%
4,5,Edmonton,"1,062,643","33,800",2.6%


<h4> From the above dataframe we now that the top 5 cities with high ethnic Italian population are Toronto, Montreal, Vancouver, Calgary and Edmonton. Our data search shall now we limited to these 5 cities.<h4>
    <h5> The next step in this process is to get data on
        <ol>
            <li> Areas called downtown in these cities and downtown latitude and longitude. </li>
            <li> Crime Rates count in each city. </li>
            <li> Venues and likes in downtown for each city.</li>
            <li> Trends in downtown for each city. </li>
            <li> Commercial property rental price for downtown in each city.</li>
        </ol>
    </h5>

In [16]:
# create a dataframe for longitude and latitide for downtown of each city
df_downtown = pd.DataFrame({'City':['Toronto','Montreal', 'Vancouver', 'Calgary','Edmonton'],'Latitude':[43.6529,45.50,49.28,51.05,53.54],'Longitude':
                          [-79.3849,-73.57,-123.12,-114.06,-113.5]})
df_downtown

,City,Latitude,Longitude
0,Toronto,43.6529,-79.3849
1,Montreal,45.5000,-73.5700
2,Vancouver,49.2800,-123.1200
3,Calgary,51.0500,-114.0600
4,Edmonton,53.5400,-113.5000


In [17]:
toronto_page = rq.get('https://en.wikipedia.org/wiki/Downtown_Toronto') # gets the toronto downtown page
print("The status of toronto page is: ",toronto_page.status_code)
toronto_soup = BeautifulSoup(toronto_page.text,'html.parser') #parse the page
# Get the list of neighborhood
toronto_list  = toronto_soup.findAll('ul')[3]
tor_rows = []
tors  = toronto_list.find_all('li')
# append to the list of neighborhood
for tor in tors:
    rows = [row.get_text() for row in tor.find_all('a')]
    tor_rows.append(rows)    
    
#convert list to dataframe
df_tor = pd.DataFrame(tor_rows)
df_tor

The status of toronto page is:  200


,0,1
0,Alexandra Park,None
1,The Annex,None
2,Cabbagetown,None
3,Chinatown,None
4,Church and Wellesley,None
5,CityPlace,None
6,Corktown,None
7,Discovery District,None
8,Distillery District,None
9,East Bayfront,None


In [18]:
# for montreal the internet search shows that it is in the borugh of Ville-Marie and there is not list of table online
montreal_page = rq.get("https://en.wikipedia.org/wiki/Ville-Marie,_Montreal")
print("The status of montreal page is: ",montreal_page.status_code)
montreal_soup = BeautifulSoup(montreal_page.content,'html.parser') #parse the page
montreal_para =montreal_soup.findAll('p')[13]
mon_rows =[]
mons = montreal_para.find_all('a')
for mon in mons:
    rows = mon.get('href')
    mon_rows.append(rows)
    
mon_rows = [w.replace('/wiki/', '') for w in mon_rows]
mon_rows = [w.replace('_', ' ') for w in mon_rows]
mon_rows = [w.replace('%27', '') for w in mon_rows]
mon_rows = [w.replace('%C3%A9', 'e') for w in mon_rows]
mon_rows = [w.replace('%C3%8E', 'l') for w in mon_rows]

df_mon = pd.DataFrame(mon_rows)
#df_mon

The status of montreal page is:  200


In [19]:
vancouver_page = rq.get("https://en.wikipedia.org/wiki/Downtown_Vancouver")
print("The status of vancouver page is: ",vancouver_page.status_code)
vancouver_soup = BeautifulSoup(vancouver_page.content,'html.parser') #parse the page
vancouver_para1 = vancouver_soup.findAll('p')[1]
vancouver_para2 = vancouver_soup.findAll('p')[2]
van_rows =[]
vans1 = vancouver_para1.find_all('a')
vans2 = vancouver_para2.find_all('a')
for van in vans1:
    rows = van.get('href')
    van_rows.append(rows)
    
for van in vans2:
    rows = van.get('href')
    van_rows.append(rows)
    
van_rows = [w.replace('/wiki/', '') for w in van_rows] # remove the wikipedia tag
van_rows = [w.replace('_', ' ') for w in van_rows] # remove the underscore
van_rows = [w.replace('#New Japantown', '') for w in van_rows]
van_rows = [w.replace('Vancouver', '') for w in van_rows] # remove vancouver as we know
van_rows = [w.replace('()', '') for w in van_rows]
van_rows = [w.replace(',', '') for w in van_rows]
df_van = pd.DataFrame(van_rows)
#df_van

The status of vancouver page is:  200


In [20]:
calgary_page = rq.get('https://en.wikipedia.org/wiki/Downtown_Calgary')
print("The status of Calgary page is: ",calgary_page.status_code)
calgary_soup = BeautifulSoup(calgary_page.content, 'html.parser')
calgary_title = calgary_soup.findAll('h3')
cal_rows=[]
for cal in calgary_title:
    rows = [row.get_text() for row in cal.find_all()]
    cal_rows.append(rows)
    #rows = [row.get_text() for row in tor.find_all('a')]
filter(None, cal_rows)
del cal_rows[5:17]
df_cal = pd.DataFrame(cal_rows)
df_cal.drop([1,2,3,4], axis =1)

The status of Calgary page is:  200


,0
0,Commercial core
1,East Village
2,West End
3,Chinatown
4,Eau Claire


In [21]:
edmonton_page = rq.get('https://en.wikipedia.org/wiki/Downtown_Edmonton')
print("The status of Edmonton page is", edmonton_page.status_code)
edmonton_soup = BeautifulSoup(edmonton_page.content, 'html.parser')
edmonton_title = edmonton_soup.findAll('h3')
ed_rows = []
for ed in edmonton_title:
    rows = [row.get_text() for row in ed.find_all()]
    ed_rows.append(rows)
    
del ed_rows[6:18]
ed_rows[5][0] = 'Ice District & Rogers Place'
ed_rows[5][1] = ''
#ed_rows[5][5] = ''
df_ed = pd.DataFrame(ed_rows)
df_ed.drop([1,2,3,4,5], axis=1, inplace=True)
#df_ed

The status of Edmonton page is 200


<h4> Now we get the crime data in each of the five cities</h4>

<h5>major crimes on toronto<h/5>

In [22]:
# get the csv file into dataframe for all cities
df_torCrime = pd.read_csv('toronto_crime.csv')
df_monCrime = pd.read_csv('montreal_crime.csv')
df_vanCrime = pd.read_csv('vancouver_crime.csv')
df_calCrime = pd.read_csv('calgary_crime.csv')
df_edCrime = pd.read_csv('edmonton_crime.csv')

In [23]:
df_monCrime.head(5)

,CATEGORIE,DATE,QUART,PDQ,X,Y,LONGITUDE,LATITUDE
0,Vol de véhicule à moteur,9/13/2018,jour,30.0,294904.159,5047548.921,-73.626778,45.567780
1,Vol de véhicule à moteur,4/30/2018,jour,30.0,294904.159,5047548.921,-73.626778,45.567780
2,Vol de véhicule à moteur,9/1/2018,nuit,7.0,290274.565,5042150.474,-73.685928,45.519122
3,Méfait,7/21/2017,jour,21.0,0.000,0.000,1.000000,1.000000
4,Méfait,7/29/2017,jour,12.0,0.000,0.000,1.000000,1.000000


<h4> The Montreal data is in French, while using Google translate API would be great idea, however since only few word need to be translated, we use replace command instead </h4>

In [24]:
df_monCrime.columns = ['Category', 'Date', 'Quarter', 'PDQ', 'X', 'Y', 'Longitude', 'Latitude']
df_monCrime = df_monCrime.replace(to_replace= "jour", value = "day")
df_monCrime = df_monCrime.replace(to_replace= "nuit", value = "night")
df_monCrime = df_monCrime.replace(to_replace= "soir", value = "evening")
df_monCrime['Category'] = df_monCrime['Category'].str.replace(u"é","e")
df_monCrime['Category'] = df_monCrime['Category'].str.replace(u"à","a")
df_monCrime = df_monCrime.replace(to_replace= "Vol de vehicule a moteur", value = "Motor Vehicle Theft")
df_monCrime = df_monCrime.replace(to_replace= "Mefait", value = "Mischeif")
df_monCrime = df_monCrime.replace(to_replace= "Vol dans / sur vehicule a moteur", value = "Flight in / on motor vehicle")
df_monCrime = df_monCrime.replace(to_replace= "Vols qualifiés", value = "Robbery")
df_monCrime = df_monCrime.replace(to_replace= "Infractions entrainant la mort", value = "Death related offences")
df_monCrime.head(5)

,Category,Date,Quarter,PDQ,X,Y,Longitude,Latitude
0,Motor Vehicle Theft,9/13/2018,day,30.0,294904.159,5047548.921,-73.626778,45.567780
1,Motor Vehicle Theft,4/30/2018,day,30.0,294904.159,5047548.921,-73.626778,45.567780
2,Motor Vehicle Theft,9/1/2018,night,7.0,290274.565,5042150.474,-73.685928,45.519122
3,Mischeif,7/21/2017,day,21.0,0.000,0.000,1.000000,1.000000
4,Mischeif,7/29/2017,day,12.0,0.000,0.000,1.000000,1.000000


In [25]:
CLIENT_ID = 'GTX25OVUZWKP3TWAT4LDWD033PMG3GDPTJ1AWLEJM4AJ2Y1L'
CLIENT_SECRET = 'NVTT3LRGATWUL0MGH2SLSTYNNUA4QUFCWZJMRAXWYAWGTLIL'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)
search_query = 'Italian'
radius = 1000 # search 1 km from the center of downtown.

Your credentails:
CLIENT_ID: GTX25OVUZWKP3TWAT4LDWD033PMG3GDPTJ1AWLEJM4AJ2Y1L
CLIENT_SECRET: NVTT3LRGATWUL0MGH2SLSTYNNUA4QUFCWZJMRAXWYAWGTLIL


In [26]:
# we get the longitude and latitude in df_downtown for each city
df_downtown

,City,Latitude,Longitude
0,Toronto,43.6529,-79.3849
1,Montreal,45.5000,-73.5700
2,Vancouver,49.2800,-123.1200
3,Calgary,51.0500,-114.0600
4,Edmonton,53.5400,-113.5000


In [27]:
#df.loc[df['B'] == 3, 'A'].iloc[0]
#for Toronto
lat_tor = df_downtown.loc[df_downtown['City'] == 'Toronto','Latitude'].iloc[0]
long_tor = df_downtown.loc[df_downtown['City'] == 'Toronto','Longitude'].iloc[0]
#for Montreal
lat_mon = df_downtown.loc[df_downtown['City'] == 'Montreal','Latitude'].iloc[0]
long_mon = df_downtown.loc[df_downtown['City'] == 'Montreal','Longitude'].iloc[0]
#for Vancouver
lat_van = df_downtown.loc[df_downtown['City'] == 'Vancouver','Latitude'].iloc[0]
long_van = df_downtown.loc[df_downtown['City'] == 'Vancouver','Longitude'].iloc[0]
#for Calgary
lat_cal = df_downtown.loc[df_downtown['City'] == 'Calgary','Latitude'].iloc[0]
long_cal = df_downtown.loc[df_downtown['City'] == 'Calgary','Longitude'].iloc[0]
#for Edmonton
lat_ed = df_downtown.loc[df_downtown['City'] == 'Edmonton','Latitude'].iloc[0]
long_ed = df_downtown.loc[df_downtown['City'] == 'Edmonton','Longitude'].iloc[0]



In [28]:
#url for each city 
#Toronto
url_tor = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat_tor, long_tor, VERSION, search_query, radius, LIMIT)
#Montreal
url_mon = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat_mon, long_mon, VERSION, search_query, radius, LIMIT)
#Vancouver
url_van = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat_van, long_van, VERSION, search_query, radius, LIMIT)
#Calgary
url_cal = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat_cal, long_cal, VERSION, search_query, radius, LIMIT)
#Edmontorn
url_ed = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat_ed, long_ed, VERSION, search_query, radius, LIMIT)


In [29]:
#get results from Foursquare
#Toronto
result_tor = rq.get(url_tor).json()

In [30]:
venues_tor = result_tor['response']['venues']
#transform into dataframe
df_downtor = json_normalize(venues_tor)
df_downtor.head(5)

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,52f6816f11d24a43115dc834,Scaddabush Italian Kitchen & Bar,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1580572703,False,"382 Yonge Street, Unit #7",Gerrard,43.658920,-79.382891,"[{'label': 'display', 'lat': 43.65892029202872...",689,M5B 1S8,CA,Toronto,ON,Canada,"[382 Yonge Street, Unit #7 (Gerrard), Toronto ...",NaN
1,5b897e92db1d81002c91df8c,Fabbrica Rustic Italian,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1580572703,False,66 Wellington St W,NaN,43.647161,-79.381691,"[{'label': 'display', 'lat': 43.647161, 'lng':...",689,M5K 1E7,CA,Toronto,ON,Canada,"[66 Wellington St W, Toronto ON M5K 1E7, Canada]",NaN
2,581cad6a7c74e15859a6f890,Scaddabush Italian Kitchen & Bar,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1580572703,False,200 Front St W,at Simcoe St,43.644737,-79.385355,"[{'label': 'display', 'lat': 43.6447367776608,...",909,M5V 3J1,CA,Toronto,ON,Canada,"[200 Front St W (at Simcoe St), Toronto ON M5V...",NaN
3,4bfc0289c3ba9521c00f9653,Italian Consulate Toronto,"[{'id': '4bf58dd8d48988d12c951735', 'name': 'E...",v-1580572703,False,136 Beverley St,Dundas Street,43.654027,-79.394104,"[{'label': 'display', 'lat': 43.65402694219784...",751,NaN,CA,Toronto,ON,Canada,"[136 Beverley St (Dundas Street), Toronto ON, ...",NaN
4,4f88cf84e4b002b90ab3b9b9,LA's Italian + Bar,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1580572703,False,NaN,NaN,43.650540,-79.384603,"[{'label': 'display', 'lat': 43.65053979517576...",263,NaN,CA,NaN,NaN,Canada,[Canada],NaN


In [31]:
#Montreal
result_mon = rq.get(url_mon).json()
#get result
venues_mon = result_mon['response']['venues']
#transform into dataframe
df_downmon = json_normalize(venues_mon)
df_downmon.head(5)

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,4f4d32b5e4b04c96fa54f81a,Italian Chamber of Commerce of Canada (ICCC) (...,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1580572704,False,"550 Sherbrooke Street West, Suite 1150",45.505453,-73.572959,"[{'label': 'display', 'lat': 45.505453, 'lng':...",649,H3A 1B9,CA,Montréal,QC,Canada,"[550 Sherbrooke Street West, Suite 1150, Montr...",NaN
1,4df02a34d4c04d0392c4e0f3,Istituto Italiano di Culturo de Montréal,[],v-1580572704,False,1200 Dr Penfield,45.502263,-73.580565,"[{'label': 'display', 'lat': 45.50226286199423...",861,NaN,CA,Montréal,QC,Canada,"[1200 Dr Penfield (Stanley), Montréal QC, Canada]",Stanley
2,4cf6d7d2e308236a4564e8ac,Consulat Général d'Italie,"[{'id': '4bf58dd8d48988d12c951735', 'name': 'E...",v-1580572704,False,"3489, rue Drummond",45.502001,-73.580976,"[{'label': 'display', 'lat': 45.50200069764191...",884,H3G 1X6,CA,Montréal,QC,Canada,"[3489, rue Drummond (coin Docteur-Penfield), M...",coin Docteur-Penfield
3,53224d8211d276c290763887,Instituto Italiano di Cultura,"[{'id': '52e81612bcbc57f1066b7a33', 'name': 'S...",v-1580572704,False,1200 Dr. Penfield,45.502457,-73.580611,"[{'label': 'display', 'lat': 45.502457, 'lng':...",871,NaN,CA,Montréal,QC,Canada,"[1200 Dr. Penfield (Stanley), Montréal QC, Can...",Stanley


In [32]:
#Vancouver
result_van = rq.get(url_van).json()
#get result
venues_van = result_van['response']['venues']
#transform into dataframe
df_downvan = json_normalize(venues_van)
df_downvan.head(5)

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4dbf5ef81e72dd48b1f1a22c,Frankies Italian Kitchen,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1580572704,False,765 Beatty Street,btw Robson and Georgia,49.278052,-123.113258,"[{'label': 'display', 'lat': 49.27805152154088...",535,V6B 2M4,CA,Vancouver,BC,Canada,"[765 Beatty Street (btw Robson and Georgia), V..."
1,4aa7d4e7f964a520724d20e3,Italian Kitchen,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1580572704,False,1037 Alberni St.,btwn Burrard & Thurlow,49.284425,-123.122189,"[{'label': 'display', 'lat': 49.28442495040121...",517,V6E 1A1,CA,Vancouver,BC,Canada,"[1037 Alberni St. (btwn Burrard & Thurlow), Va..."
2,50feceb2e4b02b4f6a0d0cb2,Sciue Italian Bakery Cafe,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1580572704,False,885 West Georgia,Hornby,49.283753,-123.119782,"[{'label': 'display', 'lat': 49.28375315195239...",418,NaN,CA,Vancouver,BC,Canada,"[885 West Georgia (Hornby), Vancouver BC, Canada]"
3,5525cd05498e7bd6f8284262,Sciue Italian Bakery And Caffee,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1580572704,False,101 Robson St,Beatty,49.277679,-123.114036,"[{'label': 'display', 'lat': 49.27767942533999...",504,NaN,CA,Vancouver,BC,Canada,"[101 Robson St (Beatty), Vancouver BC, Canada]"
4,4aa92e4df964a520525220e3,Sciué Italian Bakery Caffé,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1580572704,False,126 Davie St.,btwn Pacific Blvd. & Marinaside Crescent,49.273604,-123.121027,"[{'label': 'display', 'lat': 49.27360431864816...",715,V6Z 2Y1,CA,Vancouver,BC,Canada,[126 Davie St. (btwn Pacific Blvd. & Marinasid...


In [33]:
#Calgary
result_cal = rq.get(url_cal).json()
#get result
venues_cal = result_cal['response']['venues']
#transform into dataframe
df_downcal = json_normalize(venues_cal)
df_downcal.head(5)

,id,name,categories,referralId,hasPerk,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.postalCode
0,4bc7671b8b7c9c74b16336cf,Italian Kitchen,[],v-1580572704,False,4th Ave & 3rd St,51.052236,-114.065929,"[{'label': 'display', 'lat': 51.052236, 'lng':...",483,CA,Division No. 6,AB,Canada,"[4th Ave & 3rd St, Division No. 6 AB, Canada]",NaN,NaN
1,4be2eab1b02ec9b633ff4dc0,Italian Kitchen,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1580572704,False,NaN,51.049270,-114.069550,"[{'label': 'display', 'lat': 51.04927, 'lng': ...",673,CA,Calgary,AB,Canada,"[115-414 3 Street S.W., Calgary AB T2P 1R2, Ca...",115-414 3 Street S.W.,T2P 1R2
2,576aded5498e1e055196757a,Italian Guy,"[{'id': '56aa371be4b08b9a8d57350b', 'name': 'F...",v-1580572704,False,NaN,51.046913,-114.073569,"[{'label': 'display', 'lat': 51.046913, 'lng':...",1009,CA,NaN,NaN,Canada,[Canada],NaN,NaN
3,4ba583cff964a5205c0e39e3,Canadian Italian Club,[],v-1580572704,False,NaN,51.053685,-114.052798,"[{'label': 'display', 'lat': 51.053685, 'lng':...",649,CA,Calgary,AB,Canada,"[Calgary AB, Canada]",NaN,NaN
4,4be2116fb02ec9b623244cc0,Calgary Italian Club,"[{'id': '4bf58dd8d48988d171941735', 'name': 'E...",v-1580572704,False,NaN,51.053712,-114.053263,"[{'label': 'display', 'lat': 51.05371174872672...",626,CA,Calgary,AB,Canada,"[416 1 Avenue NE, Calgary AB T2E 0B4, Canada]",416 1 Avenue NE,T2E 0B4


In [34]:
#Edmonton
result_ed = rq.get(url_ed).json()
#get result
venues_ed = result_ed['response']['venues']
#transform into dataframe
df_downed = json_normalize(venues_ed)
df_downed.head(5)

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,4b2bce8af964a5205bbb24e3,allegro italian kitchen,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1580572704,False,10011 109 Street,53.538953,-113.507871,"[{'label': 'display', 'lat': 53.53895333608731...",533,T5J 3S8,CA,Edmonton,AB,Canada,"[10011 109 Street, Edmonton AB T5J 3S8, Canada]",NaN
1,4b16c2d9f964a5201abd23e3,Pazzo Pazzo Italian Cuisine,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1580572704,False,10016 103 Avenue NW,53.545532,-113.491903,"[{'label': 'display', 'lat': 53.54553184549108...",816,T5J 0H1,CA,Edmonton,AB,Canada,"[10016 103 Avenue NW, Edmonton AB T5J 0H1, Can...",NaN
2,57565179498edc8c3f70b84a,The Parlour Italian Kitchen,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1580572704,False,NaN,53.545475,-113.506836,"[{'label': 'display', 'lat': 53.545475, 'lng':...",758,NaN,CA,NaN,NaN,Canada,[Canada],NaN
3,4bef0e4896edc9b6443d4af2,Lit Wine Bar,"[{'id': '4bf58dd8d48988d123941735', 'name': 'W...",v-1580572704,False,10132 104 St NW,53.541659,-113.498720,"[{'label': 'display', 'lat': 53.54165907647855...",203,T5J 1A7,CA,Edmonton,AB,Canada,"[10132 104 St NW (in Icon Tower 1), Edmonton A...",in Icon Tower 1
4,5d61c6f1e1e64400084928e0,Alberta Centennial Monument,"[{'id': '4bf58dd8d48988d12d941735', 'name': 'M...",v-1580572704,False,NaN,53.535371,-113.507846,"[{'label': 'display', 'lat': 53.535371, 'lng':...",731,T5K 2L5,CA,Edmonton,AB,Canada,"[Edmonton AB T5K 2L5, Canada]",NaN


<h3> Lastly we find out the commercial rental space price in the five cities in Canada, we use linear regression to get the estimated price of the property that our client wants to rent. </h3>

In [35]:
torDowntown_page1 = rq.get('https://www.spacelist.ca/listings/on/toronto/retail/for-lease')
print('the status of toronto downtown page is:', torDowntown_page1.status_code)

the status of toronto downtown page is: 200


In [52]:
torDown_soup = BeautifulSoup(torDowntown_page1.content,'html.parser')

torDown_results = torDown_soup.findAll('div', attrs = {'class':'listings-wrapper'})

torDown_rows = []

for tor in torDown_results:
    # for square feet
    rows = [row.get_text() for row in tor.find_all('div', attrs={'class':'cell auto about'})]
    #rows = [[v for v in izip_longest(*[k.split('\n') for k in n])]] rows.split("ft")
    torDown_rows.append(rows)
    # for rent 
    rows = [row.get_text() for row in tor.find_all('div', attrs={'class':'rent'})]
    torDown_rows.append(rows)
    #for rent per month
    rows = [row.get_text() for row in tor.find_all('div', attrs={'class':'rent_per_month'})]
    torDown_rows.append(rows)
    #cell auto small-font dark-font flat-line-height right-cell-padding
    #for address
    rows = [row.get_text() for row in tor.find_all('div', attrs={'class':'cell auto small-font dark-font flat-line-height right-cell-padding'})]
    torDown_rows.append(rows)

<h4> There are 10 pages in this website for rental spaces.</h4>

In [53]:
#getting multiple urls for website search
for n in range(2,11):
    url_torDown = "https://www.spacelist.ca/listings/on/toronto/retail/for-lease/page/" + str(n) 
    #print(url_torDown)
    torDowntown_page2 = rq.get(url_torDown)
    torDown_soup2=BeautifulSoup(torDowntown_page2.content,'html.parser')

    torDown_results2 = torDown_soup2.findAll('div', attrs = {'class':'listings-wrapper'})

    for tor in torDown_results2:
        # for square feet
        rows = [row.get_text() for row in tor.find_all('div', attrs={'class':'cell auto about'})]
        #rows = [[v for v in izip_longest(*[k.split('\n') for k in n])]] rows.split("ft")
        torDown_rows[0].extend(rows)
        # for rent 
        rows = [row.get_text() for row in tor.find_all('div', attrs={'class':'rent'})]
        torDown_rows[1].extend(rows)
        #for rent per month
        rows = [row.get_text() for row in tor.find_all('div', attrs={'class':'rent_per_month'})]
        torDown_rows[2].extend(rows)
        #cell auto small-font dark-font flat-line-height right-cell-padding
        #for address
        rows = [row.get_text() for row in tor.find_all('div', attrs={'class':'cell auto small-font dark-font flat-line-height right-cell-padding'})]
        torDown_rows[3].extend(rows)
#convert list to dataframe
df_downtor = pd.DataFrame(torDown_rows)
df_downtor.transpose().tail(20)
#torDown_rows

,0,1,2,3
174,"1,800 ft²Office / Retail / Medical",Contact,Contact,872 Dundas St W
175,"2,335 ft²Office / Retail / Medical",Contact,Contact,26 Sousa Mendes St
176,"3,790 ft²Retail",$8.75/sf/yr,"$9,114/mo",1720 - 390 Bay St
177,"2,346 ft²Retail",Contact,Contact,0910 - 390 Bay St
178,"12,500 ft²Retail / Industrial / Other",$67.80/sf/yr,"$5,650/mo",21-22 - 300 Supertest Road
179,"3,101 ft²Retail",Contact,Contact,F2 - 1900 Eglinton Ave E
180,"1,000 ft²Retail / Other",Contact,Contact,1652 Queen St W
181,"1,863 ft²Retail",Contact,Contact,101 - 423 Queen St W
182,800 ft²Retail,$44.00/sf/yr,"$2,951-10,464/mo",777 Warden Ave
183,"5,968 ft²Office / Retail / Medical",$49.09/sf/yr,"$4,500/mo",197 Yonge Street


In [65]:
montDown_page1 = rq.get('https://www.spacelist.ca/listings/qc/montreal/retail/for-lease')
print("The status of montreal page is:", montDown_page1.status_code)

montDown_soup = BeautifulSoup(montDown_page1.content,'html.parser')
montDown_results = montDown_soup.findAll('div', attrs = {'class':'listings-wrapper'})
montDown_rows = []
for mont in montDown_results:
    # for square feet
    rows = [row.get_text() for row in mont.find_all('div', attrs={'class':'cell auto about'})]
    #rows = [[v for v in izip_longest(*[k.split('\n') for k in n])]] rows.split("ft")
    montDown_rows.append(rows)
    # for rent 
    rows = [row.get_text() for row in mont.find_all('div', attrs={'class':'rent'})]
    montDown_rows.append(rows)
    #for rent per month
    rows = [row.get_text() for row in mont.find_all('div', attrs={'class':'rent_per_month'})]
    montDown_rows.append(rows)
    #cell auto small-font dark-font flat-line-height right-cell-padding
    #for address
    rows = [row.get_text() for row in mont.find_all('div', attrs={'class':'cell auto small-font dark-font flat-line-height right-cell-padding'})]
    montDown_rows.append(rows)

The status of montreal page is: 200


In [66]:
for n in range(2,7):
    url_montDown = "https://www.spacelist.ca/listings/qc/montreal/retail/for-lease/page/" + str(n) 
    print(url_montDown)
    montDowntown_page2 = rq.get(url_montDown)
    montDown_soup2=BeautifulSoup(montDowntown_page2.content,'html.parser')

    montDown_results2 = montDown_soup2.findAll('div', attrs = {'class':'listings-wrapper'})

    for mont in montDown_results2:
        # for square feet
        rows = [row.get_text() for row in mont.find_all('div', attrs={'class':'cell auto about'})]
        #rows = [[v for v in izip_longest(*[k.split('\n') for k in n])]] rows.split("ft")
        montDown_rows[0].extend(rows)
        # for rent 
        rows = [row.get_text() for row in mont.find_all('div', attrs={'class':'rent'})]
        montDown_rows[1].extend(rows)
        #for rent per month
        rows = [row.get_text() for row in mont.find_all('div', attrs={'class':'rent_per_month'})]
        montDown_rows[2].extend(rows)
        #cell auto small-font dark-font flat-line-height right-cell-padding
        #for address
        rows = [row.get_text() for row in mont.find_all('div', attrs={'class':'cell auto small-font dark-font flat-line-height right-cell-padding'})]
        montDown_rows[3].extend(rows)
#convert list to dataframe
df_downmont = pd.DataFrame(montDown_rows)
df_downmont.transpose().tail(20)

https://www.spacelist.ca/listings/qc/montreal/retail/for-lease/page/2
https://www.spacelist.ca/listings/qc/montreal/retail/for-lease/page/3
https://www.spacelist.ca/listings/qc/montreal/retail/for-lease/page/4
https://www.spacelist.ca/listings/qc/montreal/retail/for-lease/page/5
https://www.spacelist.ca/listings/qc/montreal/retail/for-lease/page/6


,0,1,2,3
100,750 ft²Retail,$90.00/sf/yr,"$5,625/mo",Saint Catherine & Peel
101,750 ft²Retail,$29.60/sf/yr,"$1,850/mo",5774 Rue Sherbrooke O
102,"29,364 ft²Retail",Contact,Contact,9200 Boulevard Maurice-Duplessis
103,"5,200 ft²Retail",$23.00/sf/yr,"$9,966/mo",4025 Rue Wellington
104,"3,250 ft²Retail",$25.00/sf/yr,"$6,770/mo",1675 Avenue Dollard
105,"6,250 ft²Retail",Contact,Contact,15748 Boulevard de Pierrefonds
106,"2,200 ft²Retail",Contact,Contact,42 Rue McGill
107,"4,700 ft²Retail",Contact,Contact,7740 Boulevard St-Michel
108,"3,000 ft²Office / Retail",Contact,Contact,1280 Boulevard Marcel-Laurin
109,"5,000-10,600 ft²Office / Retail",Contact,Contact,7325 Boulevard Décarie


In [70]:
vanDown_page1 = rq.get('https://www.spacelist.ca/listings/bc/vancouver/retail/for-lease')
print("The status of montreal page is:", vanDown_page1.status_code)

vanDown_soup = BeautifulSoup(vanDown_page1.content,'html.parser')
vanDown_results = vanDown_soup.findAll('div', attrs = {'class':'listings-wrapper'})
vanDown_rows = []
for van in vanDown_results:
    # for square feet
    rows = [row.get_text() for row in van.find_all('div', attrs={'class':'cell auto about'})]
    #rows = [[v for v in izip_longest(*[k.split('\n') for k in n])]] rows.split("ft")
    vanDown_rows.append(rows)
    # for rent 
    rows = [row.get_text() for row in van.find_all('div', attrs={'class':'rent'})]
    vanDown_rows.append(rows)
    #for rent per month
    rows = [row.get_text() for row in van.find_all('div', attrs={'class':'rent_per_month'})]
    vanDown_rows.append(rows)
    #cell auto small-font dark-font flat-line-height right-cell-padding
    #for address
    rows = [row.get_text() for row in van.find_all('div', attrs={'class':'cell auto small-font dark-font flat-line-height right-cell-padding'})]
    vanDown_rows.append(rows)

The status of montreal page is: 200


In [71]:
#There are 12 pages for so lets scrape
for n in range(2,13):
    url_vanDown = "https://www.spacelist.ca/listings/bc/vancouver/retail/for-lease/page/" + str(n) 
    print(url_vanDown)
    vanDowntown_page2 = rq.get(url_vanDown)
    vanDown_soup2=BeautifulSoup(vanDowntown_page2.content,'html.parser')

    vanDown_results2 = vanDown_soup2.findAll('div', attrs = {'class':'listings-wrapper'})

    for van in vanDown_results2:
        # for square feet
        rows = [row.get_text() for row in van.find_all('div', attrs={'class':'cell auto about'})]
        #rows = [[v for v in izip_longest(*[k.split('\n') for k in n])]] rows.split("ft")
        vanDown_rows[0].extend(rows)
        # for rent 
        rows = [row.get_text() for row in van.find_all('div', attrs={'class':'rent'})]
        vanDown_rows[1].extend(rows)
        #for rent per month
        rows = [row.get_text() for row in van.find_all('div', attrs={'class':'rent_per_month'})]
        vanDown_rows[2].extend(rows)
        #cell auto small-font dark-font flat-line-height right-cell-padding
        #for address
        rows = [row.get_text() for row in van.find_all('div', attrs={'class':'cell auto small-font dark-font flat-line-height right-cell-padding'})]
        vanDown_rows[3].extend(rows)
#convert list to dataframe
df_downvan = pd.DataFrame(vanDown_rows)
df_downvan.transpose().tail(20)

https://www.spacelist.ca/listings/bc/vancouver/retail/for-lease/page/2
https://www.spacelist.ca/listings/bc/vancouver/retail/for-lease/page/3
https://www.spacelist.ca/listings/bc/vancouver/retail/for-lease/page/4
https://www.spacelist.ca/listings/bc/vancouver/retail/for-lease/page/5
https://www.spacelist.ca/listings/bc/vancouver/retail/for-lease/page/6
https://www.spacelist.ca/listings/bc/vancouver/retail/for-lease/page/7
https://www.spacelist.ca/listings/bc/vancouver/retail/for-lease/page/8
https://www.spacelist.ca/listings/bc/vancouver/retail/for-lease/page/9
https://www.spacelist.ca/listings/bc/vancouver/retail/for-lease/page/10
https://www.spacelist.ca/listings/bc/vancouver/retail/for-lease/page/11
https://www.spacelist.ca/listings/bc/vancouver/retail/for-lease/page/12


,0,1,2,3
215,"5,200 ft²Retail",$40.00/sf/yr,"$17,332/mo",1210 Granville Street
216,"4,395 ft²Retail",Contact,Contact,1391 E 33rd Ave
217,"2,575 ft²Retail",Contact,Contact,1062 Homer St
218,877 ft²Retail,Contact,Contact,4213 Main St
219,"2,655 ft²Retail",Contact,Contact,432 W Hastings St
220,"1,113 ft²Retail",Contact,Contact,2869 Commercial Dr
221,"2,860 ft²Retail",Contact,Contact,955 Granville St
222,795 ft²Retail,Contact,Contact,2935 W 4th Ave
223,851 ft²Retail,Contact,Contact,2693 W Broadway
224,"1,070 ft²Retail",Contact,Contact,518 Davie St


In [72]:
calDown_page1 = rq.get('https://www.spacelist.ca/listings/bc/vancouver/retail/for-lease')
print("The status of montreal page is:", calDown_page1.status_code)

calDown_soup = BeautifulSoup(calDown_page1.content,'html.parser')
calDown_results = calDown_soup.findAll('div', attrs = {'class':'listings-wrapper'})
calDown_rows = []
for cal in calDown_results:
    # for square feet
    rows = [row.get_text() for row in cal.find_all('div', attrs={'class':'cell auto about'})]
    calDown_rows.append(rows)
    # for rent 
    rows = [row.get_text() for row in cal.find_all('div', attrs={'class':'rent'})]
    calDown_rows.append(rows)
    #for rent per month
    rows = [row.get_text() for row in cal.find_all('div', attrs={'class':'rent_per_month'})]
    calDown_rows.append(rows)
    #for address
    rows = [row.get_text() for row in cal.find_all('div', attrs={'class':'cell auto small-font dark-font flat-line-height right-cell-padding'})]
    calDown_rows.append(rows)

The status of montreal page is: 200


In [73]:
#There are 12 pages for so lets scrape
for n in range(2,13):
    url_calDown = "https://www.spacelist.ca/listings/ab/calgary/b/c4wpdxfpqf/dt764rkwd3/retail/for-lease/page/" + str(n) 
    print(url_calDown)
    calDowntown_page2 = rq.get(url_calDown)
    calDown_soup2=BeautifulSoup(calDowntown_page2.content,'html.parser')

    calDown_results2 = calDown_soup2.findAll('div', attrs = {'class':'listings-wrapper'})

    for cal in calDown_results2:
        # for square feet
        rows = [row.get_text() for row in cal.find_all('div', attrs={'class':'cell auto about'})]
        calDown_rows[0].extend(rows)
        # for rent 
        rows = [row.get_text() for row in cal.find_all('div', attrs={'class':'rent'})]
        calDown_rows[1].extend(rows)
        #for rent per month
        rows = [row.get_text() for row in cal.find_all('div', attrs={'class':'rent_per_month'})]
        calDown_rows[2].extend(rows)
        #for address
        rows = [row.get_text() for row in cal.find_all('div', attrs={'class':'cell auto small-font dark-font flat-line-height right-cell-padding'})]
        calDown_rows[3].extend(rows)
#convert list to dataframe
df_downcal = pd.DataFrame(calDown_rows)
df_downcal.transpose().tail(20)

https://www.spacelist.ca/listings/ab/calgary/b/c4wpdxfpqf/dt764rkwd3/retail/for-lease/page/2
https://www.spacelist.ca/listings/ab/calgary/b/c4wpdxfpqf/dt764rkwd3/retail/for-lease/page/3
https://www.spacelist.ca/listings/ab/calgary/b/c4wpdxfpqf/dt764rkwd3/retail/for-lease/page/4
https://www.spacelist.ca/listings/ab/calgary/b/c4wpdxfpqf/dt764rkwd3/retail/for-lease/page/5
https://www.spacelist.ca/listings/ab/calgary/b/c4wpdxfpqf/dt764rkwd3/retail/for-lease/page/6
https://www.spacelist.ca/listings/ab/calgary/b/c4wpdxfpqf/dt764rkwd3/retail/for-lease/page/7
https://www.spacelist.ca/listings/ab/calgary/b/c4wpdxfpqf/dt764rkwd3/retail/for-lease/page/8
https://www.spacelist.ca/listings/ab/calgary/b/c4wpdxfpqf/dt764rkwd3/retail/for-lease/page/9
https://www.spacelist.ca/listings/ab/calgary/b/c4wpdxfpqf/dt764rkwd3/retail/for-lease/page/10
https://www.spacelist.ca/listings/ab/calgary/b/c4wpdxfpqf/dt764rkwd3/retail/for-lease/page/11
https://www.spacelist.ca/listings/ab/calgary/b/c4wpdxfpqf/dt764rkwd3

,0,1,2,3
212,"2,800 ft²Retail",$47.37/sf/yr,"$11,052/mo",9639 - 9639 MacLeod Trl SE
213,"3,735 ft²Retail",Contact,Contact,2980 Sunridge Way NE
214,"1,200-7,600 ft²Retail",$20.00/sf/yr,"$2,000/mo",4804 Edmonton Trl NE
215,"51,787 ft²Retail",Contact,Contact,1400 12 Ave SW
216,"1,588 ft²Retail",Contact,Contact,100 - 396 11 Ave SW
217,"1,600 ft²Retail",Contact,Contact,15566 McIvor Blvd SE
218,"2,260 ft²Retail",Contact,Contact,12024 Sarcee Trail NW
219,"764-3,368 ft²Office / Retail",Contact,Contact,8989 Macleod Trail SE
220,"2,460-4,622 ft²Retail",Contact,Contact,100 - 1301 8 St SW
221,"1,053-1,724 ft²Retail",Contact,Contact,8316 Fairmount Dr SE


In [74]:
edDown_page1 = rq.get('https://www.spacelist.ca/listings/ab/edmonton/b/c4wpdxfpqf/dt764rkwd3/retail/for-lease')
print("The status of montreal page is:", edDown_page1.status_code)

edDown_soup = BeautifulSoup(edDown_page1.content,'html.parser')
edDown_results = edDown_soup.findAll('div', attrs = {'class':'listings-wrapper'})
edDown_rows = []
for ed in edDown_results:
    # for square feet
    rows = [row.get_text() for row in ed.find_all('div', attrs={'class':'cell auto about'})]
    edDown_rows.append(rows)
    # for rent 
    rows = [row.get_text() for row in ed.find_all('div', attrs={'class':'rent'})]
    edDown_rows.append(rows)
    #for rent per month
    rows = [row.get_text() for row in ed.find_all('div', attrs={'class':'rent_per_month'})]
    edDown_rows.append(rows)
    #for address
    rows = [row.get_text() for row in ed.find_all('div', attrs={'class':'cell auto small-font dark-font flat-line-height right-cell-padding'})]
    edDown_rows.append(rows)

The status of montreal page is: 200


In [75]:
#There are 12 pages for so lets scrape
for n in range(2,9):
    url_edDown = "https://www.spacelist.ca/listings/ab/edmonton/b/c4wpdxfpqf/dt764rkwd3/retail/for-lease/page/" + str(n) 
    print(url_edDown)
    edDowntown_page2 = rq.get(url_edDown)
    edDown_soup2=BeautifulSoup(edDowntown_page2.content,'html.parser')

    edDown_results2 = edDown_soup2.findAll('div', attrs = {'class':'listings-wrapper'})

    for ed in edDown_results2:
        # for square feet
        rows = [row.get_text() for row in ed.find_all('div', attrs={'class':'cell auto about'})]
        edDown_rows[0].extend(rows)
        # for rent 
        rows = [row.get_text() for row in ed.find_all('div', attrs={'class':'rent'})]
        edDown_rows[1].extend(rows)
        #for rent per month
        rows = [row.get_text() for row in ed.find_all('div', attrs={'class':'rent_per_month'})]
        edDown_rows[2].extend(rows)
        #for address
        rows = [row.get_text() for row in ed.find_all('div', attrs={'class':'cell auto small-font dark-font flat-line-height right-cell-padding'})]
        edDown_rows[3].extend(rows)
#convert list to dataframe
df_downed = pd.DataFrame(edDown_rows)
df_downed.transpose().tail(20)

https://www.spacelist.ca/listings/ab/edmonton/b/c4wpdxfpqf/dt764rkwd3/retail/for-lease/page/2
https://www.spacelist.ca/listings/ab/edmonton/b/c4wpdxfpqf/dt764rkwd3/retail/for-lease/page/3
https://www.spacelist.ca/listings/ab/edmonton/b/c4wpdxfpqf/dt764rkwd3/retail/for-lease/page/4
https://www.spacelist.ca/listings/ab/edmonton/b/c4wpdxfpqf/dt764rkwd3/retail/for-lease/page/5
https://www.spacelist.ca/listings/ab/edmonton/b/c4wpdxfpqf/dt764rkwd3/retail/for-lease/page/6
https://www.spacelist.ca/listings/ab/edmonton/b/c4wpdxfpqf/dt764rkwd3/retail/for-lease/page/7
https://www.spacelist.ca/listings/ab/edmonton/b/c4wpdxfpqf/dt764rkwd3/retail/for-lease/page/8


,0,1,2,3
126,"1,724 ft²Retail",$33.00/sf/yr,"$4,741/mo",10546 82 Ave NW
127,"3,021 ft²Retail",Contact,Contact,745 Mistatim Way
128,"1,039-2,488 ft²Retail",Contact,Contact,11611 107 Ave NW
129,"947-2,128 ft²Retail",Contact,Contact,11018 Jasper Ave NW
130,"1,018 ft²Office / Retail",$20.70/sf/yr,"$1,756/mo",4986 92 Avenue Northwest
131,975 ft²Retail,Contact,Contact,10010 104 Street
132,"15,729 ft²Retail",Contact,Contact,17328 Stony Plain Rd
133,"1,671 ft²Retail",Contact,Contact,6825 177 Street Northwest
134,"1,637-2,200 ft²Retail",Contact,Contact,10525 Jasper Avenue NW
135,"3,287 ft²Office / Retail / Industrial",$18.76/sf/yr,"$5,138/mo",10542 - 10534 106 St NW


<h2><center>End of Week 4</center></h2>

<h3>This marks the end of week 4 assignment of the IBM Capstone project</h3>
<h4>In this assignment, we have
    <ol>
        <li>Described the problem statement as provided by the client</li>
        <li>We have collected data on: </li>
        <ul><li>population and demographics in various metropolitan cities in Canada, </li>
            <li>downtown neighborhoods in various cities in Canada </li>
            <li>Crime rate in these cities</li>
            <li>Various venues in this cities</li>
            <li>and lastly, Rental prices for retail location in these cities</li>
        </ul>
        <li>The next step would be
            <ul>
                <li>Create histogram of population and demographics of these cities</li>
                <li>based on the crime rate, cluster the data on crime and types of crime in downtown</li>
                <li>find an ideal location with low crime rate</li>
                <li>based on data from venues, get the popularity of Italian cuisine in the cities</li>
                <li>and lastly, using linear regression, estimate the rent of retail location</li>
            </ul>
        <li>So far we have done the following steps
            <ul><li>Business Understanding
                <li>Analytic Approach
                <li>Data Collection
            </ul></li>
        <li> The steps that will be followed are
            <ul><li>Data Understanding
                <li>Data Preparation (cleaning)
                <li>Modeling
                <li>Evaluation
                <li>Deployment and feedback should follow
            </ul>
        <li>Since this is a test project, Deployment and feedback are not possible</li>
        <li>A final report will be created to explain the complete project</li>
    </ol></h4>